<a href="https://colab.research.google.com/github/KHRMNKY/python-project-1/blob/main/RAG_with_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken  pypdf InstructorEmbedding faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=810426940d3f7718e1deecb05c94d9ba9cef8d9ca5805678473053089c7bea04
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
# InstructorEmbedding
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive"


Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/Documents/inek"

In [ ]:
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader(path , glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
documents

[Document(page_content=' \n \nÇİFTÇİ BROŞÜRÜ                                                                            No: 140 \n \n \nSIĞIRCILIKTA SIKÇA GÖRÜLEN  BESLEME HASTALIKLARI \n \n \nUzm. A. Mutlu UYGUR \n \nSığırcılıkta gerek yem ve yemlemeden dolay ı, gerekse farkl ı fizyolojik \ndurumlarda, artan besin madde ihtiyac ının tam kar şılanamamas ı nedeniyle veya k ısa \nzamanda a şırı tüketimler yüzünden kaynaklanan bir çok besleme hastal ıklar ı ortaya \nçıkmaktad ır. S ığırcılıkta s ıkça görülen en önemli besleme hastal ıklar ı ketosiz, \nasidosiz, süt hummas ı, üre (amonyak) zehirlenmesi ve şişmedir (timpani). \n Ketosiz:  Süt ineklerinde enerji gere ksiniminin tam olarak kar şılanamamas ı \nnedeniyle ortaya ç ıkan bir besleme hastal ığıdır. Ketosiz, genellikle yüksek verimli \nineklerde, daha çok ya şlı hayvanlarda, buza ğılamadan sonra 6-8 hafta içerisinde \nçoğunlukla 2-4. haftalarda ortaya ç ıkmaktad ır. İneklerin enerji gereksinimi \nkarşılanmad\nığı zaman süt sentezi i

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=1000,
                                               chunk_overlap=200)

texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

483

In [ ]:
texts[3]

Document(page_content='etkilemektedir.   \nKurudaki ineklerin a şırı beslenmeleri, organizmada ya ğ depolanmas ına neden \nolmakta ve do ğumdan sonra yetersiz besleme sonucunda artan enerji ihtiyac ının \nvücuttan kar şılanmas ı sonucu, ya ğlar ın parçalanmas ı ile keton maddelerinin art ışına \nneden olmaktad ır. Süt ineklerinin beslenmesinde mineral madde, iz element ve \nvitamin yetersizlikleri de ketosize neden olabilmektedir.  \nKetosize yakalanm ış olan ineklerde, süt verimi  ve yem tüketimi aniden \ndüşmekte, canl ı ağırlık kayb ı, kalp at ışlar ı ve solunumda düzensizlikler görülmektedir. \nRuminasyon (gevi ş getirme) yava şlamakta, rumen hareketleri azalmakta, yo ğun yem \ntüketiminde isteksizlik görülmektedir. Buna kar şın,  yem de\nğeri düşük yemlerin veya \nyarars ız çeşitli maddelerin tüketimi artmaktad ır. Hastal ık nedeniyle kötü kokulu, ho ş \nolmayan bir solunum ve ishal ortaya ç ıkmaktad ır. Ketosiz sinir sisteminde de', metadata={'source': '/content/gdrive/MyDrive/Do

In [ ]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="asafaya/kanarya-750m"
                                                         )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/589k [00:00<?, ?B/s]

FAISS sayesınde metnimi database de embedding hallerını indkeskli hale getiridim

In [ ]:
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

burda databasi bilgi alıcısına dönusturduk

In [ ]:
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})

burda sadece ilgili belgelerı getırmesını ıstedım bu kod satırı sart degıl

In [ ]:
docs = retriever.get_relevant_documents("Ketosiz hastalığı nedir kimlerde görülür?")

In [ ]:
docs[0]

Document(page_content='Cl.septicum) blackleg, malignant ödem ve black disease (kara hastalık) etkilidir. Polivalan \nclostridial aşılar (5 -7 suş) kullanılır.  \n \n \n \n \n \n \n \n \n \nŞekil 223.  Kaslarda koyu kırmızı ile siyah arasında değişen renk değişiklikleri ve deri \naltı bağ dokuda gaz toplanmaları (Zettl, K. 1980)', metadata={'source': '/content/gdrive/MyDrive/Documents/inek/spesifik-enfeksiyonlar-ve-yenidogan-hastaliklari.pdf', 'page': 25})

yukarda bilgi alıcımız vardı ama bu bir liste dondurdu ve bunun aslında kullanıcıya tek bır cevap vermesı lazım bu yuzden chaın kullandık ve boylece qa_chain_openai degıskenı bır sozluk yaoısında ve ıkı tane key e yanı anahtara sahıp oldu bır tane key sonuc dondururken dıgerı ılgılı dokumanı getırıyor. bu zıncır sayesınde tek bır cevap uretıldı.

In [ ]:
# depseekcoder api key
api_key= "sk-a19337c960d6455fb7f236224478e78c"
base_url = "https://api.deepseek.com/v1"

client = OpenAI(api_key=api_key, base_url=base_url)

qa_chain_deepseek = RetrievalQA.from_chain_type(
    llm=client,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
query = "Ketosiz hastalığı nedir kimlerde görülür?"
llm_response = qa_chain_deepseek(query)
print(llm_response["result"])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


NotFoundError: Error code: 404 - {'detail': 'Not Found'}